In [1]:
# https://www.youtube.com/watch?v=levekYbxauw&list=PLQVvvaa0QuDcjD5BAw2DxE6OF2tius3V3&index=8
# https://github.com/Sentdex/NNfSiX/blob/master/Python/p008-Categorical-Cross-Entropy-Loss-applied.py

import numpy as np

# Create Data
#https://cs231n.github.io/neural-networks-case-study/
# https://gist.github.com/Sentdex/454cb20ec5acf0e76ee8ab8448e6266c
# points = 100 # number of points per class
# D = 2 # dimensionality
# classes = 3 # number of classes
def spiral_data(points, classes):
    X = np.zeros((points*classes, 2))  # data matrix  # D = 2 # dimensionality
    y = np.zeros(points*classes, dtype='uint8')   #class labels
    for class_number in range(classes):
        ix = range(points*class_number, points*(class_number+1))
        r = np.linspace(0.0, 1, points)  # radius
        t = np.linspace(class_number*4, (class_number+1)*4, points) + np.random.randn(points)*0.2
        X[ix] = np.c_[r*np.sin(t*2.5), r*np.cos(t*2.5)]
        y[ix] = class_number
    return X, y

In [20]:
print(X.shape)
print(y.shape)
print(len(X))
print(len(y))
print(len(X.shape))
print(len(y.shape))

(300, 2)
(300,)
300
300
2
1


In [24]:
# We will intial weight randomly, so the dimension of Weights should be matrix(4,N) as the input is matrx(3,4)
    
class Layer_Dense:
    # we need to generate Weights and Biases
    def __init__(self, n_inputs, n_neurons): 
        self.weights = 0.1 * np.random.randn(n_inputs, n_neurons)     # 0.1 to ensure that the range of values is low ( less than 1)
        self.biases = np.zeros((1,n_neurons))   # biases we will be added to output of sum of weights so should be matrix(1,N )
        
    def forward(self,inputs):  # inputs is last ouput
        self.output = np.dot(inputs,self.weights)+self.biases

        
class Activation_Rellu:
    def forward(self, inputs): # inputs here is output of summation of wieghts and neurons
        self.output = np.maximum(0,inputs)
        
               
class Activation_Softmax:
    def forward(self, inputs):
        exp_values = np.exp(inputs - np.max(inputs, axis=1, keepdims=True))
        probabilities = exp_values / np.sum(exp_values, axis=1, keepdims=True)
        self.output = probabilities

        
class Loss:
    def calculate(self,output,y):
        sample_losses =self.forward(output,y) # will return negative_log_likelihoods from Loss_CategoricalCrossentropy class
        data_loss = np.mean(sample_losses)
        return data_loss

class Loss_CategoricalCrossentropy(Loss): # will be inhert from loss class
    def forward(self,y_pred,y_true):
        samples = len(y_pred)
        y_clipped_pred =  np.clip(y_pred, 1e-7, 1-1e-7)  # use clip() to avoid inifinty issue in case of log(0)
        # y_true (target) will be passed either in form of scalar values [1,0,1] or One hot encoding [[0 ,1],[1,0],[0,1]]
        if len(y_true.shape) == 1: # scalar values, just 1 d
            correct_confidences = y_clipped_pred[range(samples), y_true]

        elif len(y_true.shape) == 2: #One hot encoding, so it will be just muliplication of two matrix, then sum or (max value) to get one value per row or element (not our case in this data)
            correct_confidences = np.sum(y_clipped_pred*y_true, axis=1)
    
        negative_log_likelihoods = -np.log(correct_confidences)
        return negative_log_likelihoods


In [25]:
# X, y = spiral_data(samples_batch=100, classes_indata=3)
X, y = spiral_data(points=100, classes=3)

layer1= Layer_Dense(2,5)  # X has two features(neurons) as input, 5 is my desire neurons
Activation1 = Activation_Rellu()

layer2= Layer_Dense(5,4) # we choose 5 to be like previous output neurons and 4 is my desire neurons
Activation2 = Activation_Rellu()

layer3= Layer_Dense(4,3)
Activation3 = Activation_Softmax()


layer1.forward(X)
Activation1.forward(layer1.output)

layer2.forward(Activation1.output)
Activation2.forward(layer2.output)
Activation2.output.shape

layer3.forward(Activation2.output)
Activation3.forward(layer3.output)

# print(Activation3.output.shape)
# print(Activation3.output[:5])
# print(Activation3.output)
loss_function = Loss_CategoricalCrossentropy()
print(loss_function.forward(Activation3.output, y).shape)
print(loss_function.forward(Activation3.output, y)) # before mean
loss = loss_function.calculate(Activation3.output, y)


print("Loss:", loss)
#  Loss vaues are very high for all values and also mean value

(300,)
[1.09861229 1.09865032 1.09866862 1.09872078 1.09866165 1.09861954
 1.0988573  1.09868236 1.09862676 1.09862847 1.09865048 1.09904661
 1.0986552  1.09866084 1.09866307 1.09866803 1.09867357 1.09867545
 1.09868099 1.09868423 1.09868881 1.09868161 1.09869618 1.09865298
 1.0986983  1.09869271 1.09869906 1.09868616 1.09871985 1.09866875
 1.0986401  1.0986332  1.09867029 1.09862397 1.0986537  1.09865145
 1.09867717 1.09863667 1.09862742 1.09865132 1.09862804 1.09891994
 1.09863895 1.09893478 1.09863034 1.09948022 1.09879556 1.09954714
 1.09950906 1.09905835 1.09863882 1.10023042 1.09874936 1.10069928
 1.09898447 1.0998798  1.10053085 1.10075312 1.10090537 1.10095385
 1.10107236 1.10111671 1.10061322 1.10117157 1.10094437 1.10106285
 1.09905835 1.09956313 1.09899492 1.09894075 1.09934773 1.10145232
 1.10076425 1.10073994 1.09887557 1.09914107 1.09890704 1.09891223
 1.09891681 1.09882834 1.09892658 1.09892887 1.09892675 1.09877101
 1.09883229 1.09884272 1.09885881 1.09888339 1.09893956